In [35]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import RNN
import pandas as pd
import numpy as np

torch.manual_seed(1)

In [23]:
lstm = nn.LSTM(20, 20)  # Input dim is 20, output dim is 20
index = [15,28,29,40,41,81,82,84,85,86,98,106,107,108,109,132,137,141]
data = {}
for i in index:
    data['%s'%i] = pd.read_csv('data_new/4/biwi_%s.txt'%i, 
                               header = None, names = ['frameNb','id', 'x','y'],delimiter=' ')


In [53]:
a = 0
inputs = np.zeros([len(index),40])
for i in index:
    inputs[a,:] = np.array(data['%s'%i].loc[0:,['x','y']]).reshape(1,40)[0]
    a +=1
    
inputs = torch.from_numpy(inputs)

In [56]:
inputs.shape

torch.Size([18, 40])